## This is not an assignment, I'm just playing around with rigidity matrices

In [1]:
import numpy as np

Take a 3x4 2D object with bars:

    X -- X -- X -- X
    |    |    |    |
    X -- X -- X -- X
    |    |    |    |
    X -- X -- X -- X
    
This has $d = 2$, $N_s = 12$, $N_b = 17$, with 3 trivial zero modes.

There should be $dN_s - N_b$ = 7 zero modes, 4 of which should be non-trivial.

In [2]:
R = np.zeros ((17, 24))
# rows: stresses - 9 horizontal, then 8 vertical
# columns: 12 X displacements followed by 12 Y displacements

# add horizontal stresses
for rowoffset, coloffset, i in zip ([0,0,0,0,3,3,3,3,6,6,6,6], [0,0,0,0,4,4,4,4,8,8,8,8], range(12)):
    relative_i = i - coloffset
    if relative_i != 0: # this is not the left edge
        R[rowoffset+relative_i-1, i] = +1
    if relative_i != 3: # this is not the right edge
        R[rowoffset+relative_i, i] = -1
    
# add vertical stresses
for rowoffset, coloffset, i_from0 in zip ([9,9,9,9,13,13,13,13,17,17,17,17], [0,0,0,0,4,4,4,4,8,8,8,8], range(12)):
    relative_i = i_from0 - coloffset
    i = i_from0 + 12
    if rowoffset != 9: # this is not the top edge
        R[rowoffset+relative_i-4, i] = +1
    if rowoffset != 17: # this is not the bottom edge
        R[rowoffset+relative_i, i] = -1
        
# print rigidity
for row in R:
    print(''.join('-.+'[int(np.sign(r))+1] for r in row) + '\n')

-+......................

.-+.....................

..-+....................

....-+..................

.....-+.................

......-+................

........-+..............

.........-+.............

..........-+............

............-...+.......

.............-...+......

..............-...+.....

...............-...+....

................-...+...

.................-...+..

..................-...+.

...................-...+



Now I can construct a square matrix calculating the restoring forces when forces are applied to the points.

In [3]:
restoring_force = -R.T.dot(R)

In [4]:
w, v = np.linalg.eig (restoring_force)

In [5]:
np.count_nonzero (np.abs (w) < 1e-15)

7

Okay, that's 7 zero modes. Great!

Let's just check that the translational modes are eigenvectors.

In [6]:
x_trans = np.array([1]*12 + [0] * 12)
y_trans = np.array([0]*12 + [1] * 12)

In [7]:
R.dot(x_trans)

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.])

In [8]:
R.dot(y_trans)

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.])

Okay. Now let's remove those extra modes by adding extra contraints. Let's just add one, somewhere.

    X -- X -- X -- X
    |    | \\ |    |
    X -- X -- X -- X
    |    |    |    |
    X -- X -- X -- X
    
This has $d = 2$, $N_s = 12$, $N_b = 18$, with 3 trivial zero modes.

There should be one zero mode fewer...

In [9]:
R2 = np.zeros ((18, 24))

R2[:17, :] = R
R2[17,1] = R2[17,12+1] = +np.sqrt(2)
R2[17,4+2] = R2[17,12+4+2] = -np.sqrt(2)

In [10]:
restoring_force2 = -R2.T.dot(R2)
w2, v2 = np.linalg.eig (restoring_force2)
np.count_nonzero(np.abs(w2) < 1e-15)

6

Okay. I'm reasonably happy with that. Let's just take this just 4 steps further:

    X -- X -- X -- X
    | // | \\ | // |
    X -- X -- X -- X
    | \\ | // |    |
    X -- X -- X -- X

In [11]:
R3 = np.zeros ((22, 24))
R3[:18,:] = R2
R3[18,1] = R3[18,12+1] = -np.sqrt(2)
R3[18,4+0] = R3[18,12+4+0] = +np.sqrt(2)
R3[18,3] = R3[18,12+3] = -np.sqrt(2)
R3[18,4+2] = R3[18,12+4+2] = +np.sqrt(2)
R3[18,4+2] = R3[18,12+4+2] = -np.sqrt(2)
R3[18,8+1] = R3[18,12+8+1] = +np.sqrt(2)
R3[18,4+0] = R3[18,12+4+0] = +np.sqrt(2)
R3[18,8+1] = R3[18,12+8+1] = -np.sqrt(2)

In [12]:
restoring_force3 = -R3.T.dot(R3)
w3, v3 = np.linalg.eig (restoring_force3)
np.count_nonzero(np.abs(w3) < 1e-15)

5

Apparently it's not so easy to find the right bonds without simply increasing self-stress.